In [ ]:
from google.colab import drive
drive.mount('/content/drive')





In [ ]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,Conv1D,MaxPooling1D, GlobalMaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from tensorflow.keras.layers import Conv2D
import os

In [ ]:

x=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NATOPS/X1_TRAIN.csv')
y=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NATOPS/Y1_TRAIN.csv')
z=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NATOPS/Z1_TRAIN.csv')
#x,y 에 라벨링 제거
x=x.drop(['classAttribute'],axis=1)
y=y.drop(['classAttribute'],axis=1)

length=len(x)
df_train=pd.DataFrame()
df_train=pd.concat([x,y,z],axis=1,ignore_index=True)
df_train.rename(columns={153:'attribute_wr'},inplace=True)


In [ ]:
x=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NATOPS/X1_TEST.csv')
y=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NATOPS/Y1_TEST.csv')
z=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NATOPS/Z1_TEST.csv')
#x,y 에 라벨링 제거
x=x.drop(['classAttribute'],axis=1)
y=y.drop(['classAttribute'],axis=1)

length=len(x)
df_test=pd.DataFrame()
df_test=pd.concat([x,y,z],axis=1,ignore_index=True)
df_test.rename(columns={153:'attribute_wr'},inplace=True)

In [ ]:
def encode(train):
    label_encoder = LabelEncoder().fit(train.attribute_wr)
    labels = label_encoder.transform(train.attribute_wr)
    classes = list(label_encoder.classes_)

    train = train.drop(['attribute_wr'], axis=1)

    return train, labels, classes

df_train, labels, classes = encode(df_train)


In [ ]:
def encode_test(test):
    label_encoder = LabelEncoder().fit(test.attribute_wr)
    labels = label_encoder.transform(test.attribute_wr)
    classes = list(label_encoder.classes_)

    test = test.drop(['attribute_wr'], axis=1)
    return test,labels, classes

df_test, y_test,classes_test=encode_test(df_test)


In [ ]:
scaler=StandardScaler().fit(df_train.values)
scaled_train=scaler.transform(df_train.values)
scaled_test=scaler.transform(df_test.values)

X_test=scaled_test
# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(df_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]

In [ ]:
nb_features = 51 # number of features per features type (shape, texture, margin)   
nb_class = len(classes)
# reshape train data
X_train_r = np.zeros((len(X_train), nb_features,3))
X_train_r[:,:,0] = X_train[:, :nb_features]
X_train_r[:,:,1] = X_train[:, nb_features:102]
X_train_r[:,:,2] = X_train[:, 102:]

In [ ]:
X_valid_r = np.zeros((len(X_valid), nb_features,3))
X_valid_r[:,:,0] = X_valid[:, :nb_features]
X_valid_r[:,:,1] = X_valid[:, nb_features:102]
X_valid_r[:,:,2] = X_valid[:, 102:]

In [ ]:
X_test_r=np.zeros((len(X_test),nb_features,3))
X_test_r[:,:,0]=X_test[:,0:nb_features]
X_test_r[:,:,1]=X_test[:,nb_features:nb_features+51]
X_test_r[:,:,2]=X_test[:,nb_features+51:]


In [ ]:
model = Sequential()
model.add(Conv1D(filters=216,kernel_size=3,strides=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(MaxPooling1D())

model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dense(216, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

In [ ]:
y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)
y_test=np_utils.to_categorical(y_test,nb_class)
sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

os.chdir('/content/drive/My Drive/NATOPS_ex/model16')

from datetime import datetime
dt = datetime.now()
start=dt
print(start)
hist=model.fit(X_train_r, y_train,validation_data=(X_valid_r,y_valid), epochs=30, batch_size=8)
dt = datetime.now()
end=dt
print(end)
train_t=end-start

print("Training time: {} ms".format(train_t))

In [ ]:
from matplotlib import pyplot as plt
# 모델 평가
print('\n-- Evalutate --')
scores = model.evaluate(X_test_r, y_test, verbose=1)

print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))


# 학습과정 그래프
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([-0.2, 1.2])

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylim([-0.2, 1.2])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

fig = plt.gcf()
plt.show()



In [ ]:

# 모델 저장
model.save('5.h5')
fig.savefig('5.png', bbox_inches='tight')
f=open('time.txt','a')

f.write("5 Accuracy: {}% \n".format(scores[1]*100))
f.write("Train time: {}\t".format(train_t))
f.write("loss: {} \n\n".format(scores[0]))
f.close()